# **ЗАДАНИЕ 1** - СЛОИ

## 1. Импорты

In [1]:
import torch
import math
import unittest

## 2. Linear

In [2]:
class MyLinear:
    def __init__(self, in_features, out_features, bias=True):
        limit = math.sqrt(6 / (in_features + out_features))
        self.W = torch.empty(out_features, in_features).uniform_(-limit, limit)
        self.b = torch.zeros(out_features) if bias else None
        self.bias = bias

    def forward(self, x):
        # x: (batch, in_features)
        y = x @ self.W.T
        if self.bias:
            y = y + self.b
        return y

    def __call__(self, x):
        return self.forward(x)


## 3. BatchNorm

In [3]:
class MyBatchNorm:
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        self.gamma = torch.ones(num_features)
        self.beta = torch.zeros(num_features)
        self.running_mean = torch.zeros(num_features)
        self.running_var = torch.ones(num_features)
        self.eps = eps
        self.momentum = momentum
        self.training = True

    def forward(self, x):
        if self.training:
            batch_mean = x.mean(dim=0)
            batch_var = x.var(dim=0, unbiased=False)
            x_hat = (x - batch_mean) / torch.sqrt(batch_var + self.eps)
            out = self.gamma * x_hat + self.beta

            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * batch_mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * batch_var
        else:
            x_hat = (x - self.running_mean) / torch.sqrt(self.running_var + self.eps)
            out = self.gamma * x_hat + self.beta
        return out

    def __call__(self, x):
        return self.forward(x)


## 4. Dropout

In [4]:
class MyDropout:
    def __init__(self, p=0.5):
        assert 0 <= p < 1, "Dropout probability must be in [0,1)"
        self.p = p
        self.training = True

    def forward(self, x):
        if self.training:
            mask = (torch.rand_like(x) > self.p).float()
            return mask * x / (1 - self.p)
        else:
            return x

    def __call__(self, x):
        return self.forward(x)


## 5. ReLU & Sigmoid

In [5]:
class MyReLU:
    def forward(self, x):
        return torch.maximum(x, torch.zeros_like(x))

    def __call__(self, x):
        return self.forward(x)


class MySigmoid:
    def forward(self, x):
        return 1 / (1 + torch.exp(-x))

    def __call__(self, x):
        return self.forward(x)


## 6. Тесты

In [6]:
class TestManualLayers(unittest.TestCase):
    def test_linear(self):
        layer = MyLinear(3, 2)
        x = torch.randn(4, 3)
        y = layer(x)
        self.assertEqual(y.shape, (4, 2))

    def test_batchnorm_train_eval(self):
        bn = MyBatchNorm(3)
        x = torch.randn(5, 3)
        y_train = bn(x)
        bn.training = False
        y_eval = bn(x)
        self.assertEqual(y_train.shape, y_eval.shape)

    def test_dropout(self):
        layer = MyDropout(p=0.5)
        x = torch.ones(1000)
        y = layer(x)
        zeros_ratio = (y == 0).float().mean().item()
        self.assertTrue(0.4 <= zeros_ratio <= 0.6)
        layer.training = False
        y_eval = layer(x)
        self.assertTrue(torch.allclose(y_eval, x))

    def test_relu(self):
        act = MyReLU()
        x = torch.tensor([-1.0, 0.0, 2.0])
        y = act(x)
        self.assertTrue(torch.allclose(y, torch.tensor([0.0, 0.0, 2.0])))

    def test_sigmoid(self):
        act = MySigmoid()
        x = torch.tensor([-1.0, 0.0, 1.0])
        y = act(x)
        expected = 1 / (1 + torch.exp(-x))
        self.assertTrue(torch.allclose(y, expected, atol=1e-5))


if __name__ == "__main__":
    unittest.main(argv=[''], exit=False)


.....
----------------------------------------------------------------------
Ran 5 tests in 0.113s

OK
